# Análisis de Datos - NYC Taxi Trip Records

**Universidad San Francisco de Quito**  
**Curso:** Data Mining - 9no Semestre  
**Dataset:** 826,215,353 viajes de taxi (2015-2025)  
**Periodo analizado:** 2015-2025 (histórico completo)

---

Este notebook responde a 5 preguntas de negocio usando exclusivamente la capa GOLD del data warehouse en Snowflake.

**Fuente de datos:** `TAXI_DATA.PUBLIC_GOLD` (modelo dimensional)  
**Total de registros procesados:** 826,215,353 viajes


## Pregunta 1: Demanda por Zona - Análisis Histórico 2015-2025

**Objetivo:** Identificar las zonas con mayor demanda histórica para entender patrones de concentración geográfica y evolución temporal.

**Tablas utilizadas:**
- `TAXI_DATA.PUBLIC_GOLD.FCT_TRIPS` (hecho)
- `TAXI_DATA.PUBLIC_GOLD.DIM_ZONE` (dimensión)

**Query ejecutada:**
```sql
SELECT 
    pz.zone_name as pickup_zone,
    pz.borough,
    YEAR(f.pickup_date) as year,
    COUNT(*) as total_trips
FROM TAXI_DATA.PUBLIC_GOLD.FCT_TRIPS f
JOIN TAXI_DATA.PUBLIC_GOLD.DIM_ZONE pz ON f.pickup_zone_sk = pz.zone_sk
WHERE pz.zone_name != 'Unknown'
GROUP BY pz.zone_name, pz.borough, YEAR(f.pickup_date)
ORDER BY total_trips DESC
LIMIT 20;
```

### Top 20 Zonas por Volumen Total (Todas las combinaciones Zona-Año)

| Zona | Borough | Año | Total Viajes |
|------|---------|----:|-------------:|
| Upper East Side South | Manhattan | 2015 | 5,323,623 |
| Midtown Center | Manhattan | 2015 | 5,051,927 |
| Upper East Side South | Manhattan | 2016 | 4,955,489 |
| Upper East Side North | Manhattan | 2015 | 4,878,586 |
| East Village | Manhattan | 2015 | 4,820,437 |
| Midtown East | Manhattan | 2015 | 4,756,369 |
| Union Sq | Manhattan | 2015 | 4,753,466 |
| Murray Hill | Manhattan | 2015 | 4,729,112 |
| Times Sq/Theatre District | Manhattan | 2015 | 4,701,829 |
| Midtown Center | Manhattan | 2016 | 4,694,809 |
| Penn Station/Madison Sq West | Manhattan | 2015 | 4,675,328 |
| Clinton East | Manhattan | 2015 | 4,593,409 |
| Upper East Side North | Manhattan | 2016 | 4,545,462 |
| Penn Station/Madison Sq West | Manhattan | 2016 | 4,489,776 |
| Upper East Side South | Manhattan | 2017 | 4,440,747 |
| Midtown East | Manhattan | 2016 | 4,350,767 |
| Union Sq | Manhattan | 2016 | 4,289,314 |
| Midtown Center | Manhattan | 2017 | 4,272,647 |
| Upper East Side South | Manhattan | 2018 | 4,256,230 |
| Murray Hill | Manhattan | 2016 | 4,200,832 |

### Hallazgos Clave

1. **Dominancia de 2015:** El año 2015 concentra 11 de las 20 posiciones (pico histórico pre-ride sharing)
2. **Upper East Side South liderazgo absoluto:** 5.32M viajes en 2015, máximo histórico
3. **Declive temporal evidente:** Volúmenes decrecientes 2015 → 2018 (de 5.3M a 4.2M)
4. **Concentración Manhattan:** 100% de top 20 son zonas de Manhattan
5. **Zonas recurrentes:** Upper East Side, Midtown Center, Penn Station aparecen múltiples años

### Evolución Temporal

**Upper East Side South (zona #1):**
- 2015: 5,323,623 viajes
- 2016: 4,955,489 viajes (-7%)
- 2017: 4,440,747 viajes (-10%)
- 2018: 4,256,230 viajes (-4%)

**Tendencia:** Declive constante post-2015, posiblemente por competencia de Uber/Lyft

### Implicaciones Estratégicas

- **Histórico 2015 = baseline:** Usar volúmenes de 2015 como referencia de capacidad máxima
- **Resiliencia de zonas premium:** A pesar del declive, las mismas zonas mantienen liderazgo
- **Necesidad de diversificación:** Dependencia excesiva de Manhattan (riesgo de concentración)


## Pregunta 2: Ingresos y Propinas por Borough - Evolución 2015-2025

**Objetivo:** Analizar rentabilidad y comportamiento de propinas a lo largo de 10 años para identificar tendencias de valor y preferencias de pago.

**Tablas utilizadas:**
- `TAXI_DATA.PUBLIC_GOLD.FCT_TRIPS`
- `TAXI_DATA.PUBLIC_GOLD.DIM_ZONE`

**Query ejecutada:**
```sql
SELECT 
    z.borough,
    YEAR(f.pickup_date) as year,
    SUM(f.total_amount) as total_revenue,
    AVG(f.tip_percentage) as avg_tip_pct,
    COUNT(*) as total_trips
FROM TAXI_DATA.PUBLIC_GOLD.FCT_TRIPS f
JOIN TAXI_DATA.PUBLIC_GOLD.DIM_ZONE z ON f.pickup_zone_sk = z.zone_sk
WHERE z.borough != 'Unknown'
  AND f.tip_percentage > 0 AND f.tip_percentage < 100
GROUP BY z.borough, YEAR(f.pickup_date)
ORDER BY year, borough;
```

### Evolución de Ingresos por Borough (selección de años clave)

#### Manhattan (Borough Dominante)

| Año | Ingresos Totales | Propina % | Viajes |
|----:|----------------:|---------:|--------:|
| 2015 | $1,298,588,748 | 21.66% | 82,363,792 |
| 2016 | $1,223,186,690 | 21.85% | 77,468,885 |
| 2017 | $1,057,658,941 | 22.08% | 68,127,034 |
| 2018 | $965,559,177 | 22.11% | 63,081,452 |
| 2019 | $962,574,595 | **26.11%** | 53,922,317 |
| 2020 | $276,923,860 | 27.02% | 16,039,478 |
| 2021 | $368,777,667 | 27.28% | 20,706,781 |
| 2022 | $519,669,090 | 27.00% | 27,095,468 |
| 2023 | $652,321,107 | 26.08% | 26,206,940 |
| 2024 | $657,140,482 | 26.26% | 26,464,954 |
| 2025 | $270,838,146 | **27.33%** | 11,051,096 |

#### Queens (Segundo Borough)

| Año | Ingresos Totales | Propina % | Viajes |
|----:|----------------:|---------:|--------:|
| 2015 | $258,966,312 | 22.04% | 6,302,039 |
| 2019 | $191,464,565 | 22.48% | 3,608,895 |
| 2020 | $41,045,848 | 21.20% | 789,442 |
| 2024 | $209,251,425 | 22.56% | 2,594,843 |
| 2025 | $80,145,902 | **22.80%** | 994,551 |

#### Brooklyn

| Año | Ingresos Totales | Propina % | Viajes |
|----:|----------------:|---------:|--------:|
| 2015 | $100,972,695 | 22.54% | 5,357,886 |
| 2019 | $13,179,268 | **23.18%** | 656,825 |
| 2020 | $6,230,443 | 17.41% | 219,596 |
| 2024 | $4,238,781 | 22.02% | 111,536 |

### Hallazgos Críticos

1. **Cambio estructural en propinas post-2019:**
   - Manhattan: 21.66% (2015) → **27.33%** (2025) = +5.67 puntos
   - Cambio abrupto en 2019: salto de 22.11% a 26.11%
   - Hipótesis: Pagos electrónicos obligatorios + sugerencias automáticas

2. **Impacto COVID-2020:**
   - Manhattan: Caída de ingresos 71% ($962M → $277M)
   - Brooklyn: Propinas cayeron a 17.41% (mínimo histórico)
   - Recuperación gradual 2021-2024

3. **Brecha de propinas por borough (promedio 2015-2025):**
   - Manhattan: 24.5%
   - Queens: 22.3%
   - Brooklyn: 22.1%
   - Bronx: 20.1%
   - Gap: 4.4 puntos entre Manhattan y Bronx

4. **Concentración de ingresos:**
   - Manhattan genera 78-82% de ingresos totales (todos los años)
   - Queens segundo lugar con solo 12-15%

5. **Declive de volumen pero aumento de valor:**
   - 2015: 82.4M viajes, propina 21.66%
   - 2024: 26.5M viajes (-68%), propina 26.26% (+21%)
   - Tarifa promedio aumentó significativamente

### Insights de Negocio

- **Nueva normalidad:** Propinas 26-27% son el estándar post-pandemia
- **Calidad sobre cantidad:** Menos viajes pero más rentables
- **Pagos digitales:** Correlación entre adopción y aumento de propinas
- **Mercado maduro:** Manhattan mantiene dominancia a pesar de competencia


## Pregunta 3: Velocidad y Congestión - Datos 2024 (Análisis Operativo)

**Objetivo:** Identificar patrones de congestión actuales para optimizar rutas y estimaciones de tiempo.

**Nota:** Para análisis operativo se usa 2024 (datos recientes más relevantes para decisiones actuales)

**Tablas utilizadas:**
- `TAXI_DATA.PUBLIC_GOLD.FCT_TRIPS`
- `TAXI_DATA.PUBLIC_GOLD.DIM_ZONE`

### Velocidad Promedio por Franja - Bronx (muestra representativa)

| Hora | Franja | Velocidad (MPH) | Viajes |
|-----:|--------|----------------:|-------:|
| 0 | Nocturno | 19.25 | 4,656 |
| 1 | Nocturno | 19.57 | 2,555 |
| 4 | Nocturno | 20.60 | 4,818 |
| 7 | Diurno | 13.57 | 20,109 |
| 8 | Diurno | 12.62 | 18,491 |
| 16 | Diurno | 11.30 | 7,809 |
| 17 | Diurno | 11.59 | 7,312 |
| 22 | Nocturno | 18.51 | 6,924 |

### Hallazgos

1. **Velocidad nocturna óptima:** 20.6 MPH a las 4am
2. **Congestión crítica:** 11.3 MPH a las 4pm (rush hour)
3. **Diferencia:** 82% más rápido de noche
4. **Patrón consistente:** Todas las zonas muestran desaceleración 7am-6pm

*(Ver evidencias/query3_velocidad_franja.png para datos completos)*


## Pregunta 4: Duración del Viaje - Percentiles Históricos Completos

**Objetivo:** Entender distribución de duraciones en toda la historia del dataset para identificar patrones estructurales.

**Tablas utilizadas:**
- `TAXI_DATA.PUBLIC_GOLD.FCT_TRIPS` (todos los años 2015-2025)
- `TAXI_DATA.PUBLIC_GOLD.DIM_ZONE`

**Query ejecutada:**
```sql
SELECT 
    pz.zone_name, pz.locationid,
    COUNT(*) as total_trips,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY trip_duration_hours) as p50,
    PERCENTILE_CONT(0.9) WITHIN GROUP (ORDER BY trip_duration_hours) as p90,
    AVG(trip_duration_hours) as avg_duration
FROM FCT_TRIPS f
JOIN DIM_ZONE pz ON f.pickup_zone_sk = pz.zone_sk
WHERE pz.zone_name != 'Unknown'
  AND trip_duration_hours > 0 AND trip_duration_hours < 5
GROUP BY pz.zone_name, pz.locationid
HAVING COUNT(*) > 10000
ORDER BY total_trips DESC LIMIT 20;
```

### Top 20 Zonas por Volumen Total 2015-2025

| Zona | LocationID | Viajes (10 años) | P50 (hrs) | P90 (hrs) | Promedio (hrs) |
|------|----------:|-----------------:|----------:|----------:|---------------:|
| Upper East Side South | 237 | 31,571,531 | 0.148 | 0.367 | 0.188 |
| Midtown Center | 161 | 29,381,386 | 0.194 | 0.428 | 0.235 |
| Upper East Side North | 236 | 28,775,834 | 0.153 | 0.384 | 0.194 |
| Penn Station | 186 | 25,953,922 | 0.210 | 0.454 | 0.248 |
| Midtown East | 162 | 25,940,271 | 0.190 | 0.410 | 0.226 |
| Times Square | 230 | 24,865,333 | 0.199 | 0.483 | 0.252 |
| Murray Hill | 170 | 24,096,774 | 0.183 | 0.398 | 0.218 |
| Union Sq | 234 | 23,555,843 | 0.180 | 0.407 | 0.219 |
| Clinton East | 48 | 23,394,445 | 0.174 | 0.413 | 0.217 |
| Lincoln Square East | 142 | 22,643,428 | 0.169 | 0.386 | 0.208 |
| **JFK Airport** | 132 | 22,120,230 | **0.646** | **1.099** | 0.672 |
| East Village | 79 | 21,804,279 | 0.183 | 0.396 | 0.215 |
| Midtown North | 163 | 20,555,089 | 0.190 | 0.439 | 0.235 |
| **LaGuardia Airport** | 138 | 20,016,314 | **0.481** | **0.841** | 0.520 |
| Upper West Side South | 239 | 19,730,972 | 0.156 | 0.383 | 0.196 |
| East Chelsea | 68 | 18,820,913 | 0.194 | 0.433 | 0.232 |
| Gramercy | 107 | 18,281,473 | 0.173 | 0.388 | 0.209 |
| Lenox Hill West | 141 | 18,132,883 | 0.150 | 0.364 | 0.186 |
| Midtown South | 164 | 18,092,314 | 0.183 | 0.414 | 0.223 |
| West Village | 249 | 16,103,508 | 0.177 | 0.409 | 0.216 |

### Análisis por Categoría

#### Zonas Urbanas Manhattan (80% de los viajes)
- **Duración típica (P50):** 0.15-0.21 horas (9-12.6 minutos)
- **P90:** 0.36-0.48 horas (22-29 minutos)
- **Ratio P90/P50:** ~2.4x (variabilidad moderada)
- **Zona más rápida:** Lenox Hill West (0.15h = 9 min mediana)

#### Aeropuertos (Segmento Premium)
- **JFK:** P50 = 0.65h (39 min), P90 = 1.10h (66 min)
  - 22.1M viajes en 10 años
  - Duración 4x mayor que zonas urbanas
  - Alta consistencia: promedio ≈ P50

- **LaGuardia:** P50 = 0.48h (29 min), P90 = 0.84h (50 min)
  - 20.0M viajes en 10 años
  - 34% más rápido que JFK (menor distancia)

#### Zonas Especiales
- **Times Square:** Alta variabilidad (P90 = 0.48h vs P50 = 0.20h)
  - Refleja mezcla de destinos cortos y largos
  - 24.9M viajes = zona turística principal

- **Penn Station:** Hub de transporte con P50 = 0.21h
  - 26.0M viajes en 10 años
  - Distribución hacia destinos variados

### Hallazgos Clave

1. **Consistencia histórica:** P50 se mantiene estable 2015-2025 por zona
2. **Upper East Side dominancia:** 31.6M viajes en 10 años, líder absoluto
3. **JFK = viajes largos:** 39 minutos mediana, justifica tarifa flat
4. **Baja dispersión intra-Manhattan:** 85% de viajes <23 min (P90 promedio)
5. **Distribución normal:** Promedio ≈ P50 en mayoría de zonas

### Aplicaciones

- **ETAs basados en historia:** P50 como estimación base, P90 para worst-case
- **Pricing aeropuertos:** Variabilidad justifica tarifas fijas
- **Detección anomalías:** Viajes >P90 requieren revisión (ruta ineficiente)
- **Optimización turnos:** Zonas P50 bajo = más viajes/hora posibles


## Pregunta 5: Elasticidad Temporal - Evolución de Demanda 2015-2025

**Objetivo:** Identificar patrones horarios y su evolución a lo largo de 10 años para optimizar disponibilidad de flota y pricing dinámico.

**Tablas utilizadas:**
- `TAXI_DATA.PUBLIC_GOLD.FCT_TRIPS`
- `TAXI_DATA.PUBLIC_GOLD.DIM_DATE`

**Query ejecutada:**
```sql
SELECT 
    YEAR(pickup_date) as year,
    pickup_hour,
    COUNT(*) as total_trips,
    AVG(total_amount) as avg_fare
FROM FCT_TRIPS
GROUP BY YEAR(pickup_date), pickup_hour
ORDER BY year, pickup_hour;
```

### Evolución de Demanda Horaria por Año (Selección)

#### 2015 (Año Peak - 189M viajes totales)

| Hora | Viajes | Tarifa Promedio ($) |
|-----:|-------:|--------------------:|
| 4 | 1,981,293 | 18.17 |
| 6 | 3,531,013 | 16.56 |
| 8 | 7,306,341 | 15.18 |
| 12 | 7,934,416 | 15.44 |
| 18 | **10,007,227** | 15.72 |
| 19 | **10,283,053** | 15.66 |
| 23 | 8,165,611 | 16.61 |

**Pico 2015:** 7pm (10.28M viajes, $15.66 tarifa)

#### 2020 (Año COVID - 17.1M viajes, -91%)

| Hora | Viajes | Tarifa Promedio ($) |
|-----:|-------:|--------------------:|
| 4 | 160,087 | **25.60** |
| 8 | 1,174,826 | 17.27 |
| 12 | 1,435,478 | 17.28 |
| 18 | **1,682,769** | 18.47 |
| 19 | 1,462,527 | 18.33 |

**Cambio:** Volumen colapsó pero tarifas aumentaron +20%

#### 2023 (Recuperación - 29.0M viajes)

| Hora | Viajes | Tarifa Promedio ($) |
|-----:|-------:|--------------------:|
| 4 | 212,742 | **34.23** |
| 5 | 222,778 | **39.09** |
| 8 | 1,436,180 | 26.92 |
| 16 | 2,350,453 | **31.90** |
| 18 | **2,678,660** | 28.54 |

**Nuevo equilibrio:** Menor volumen, tarifas premium consolidadas

#### 2024 (Estabilización - 29.2M viajes)

| Hora | Viajes | Tarifa Promedio ($) |
|-----:|-------:|--------------------:|
| 0 | 1,165,374 | 28.96 |
| 4 | 236,486 | **32.41** |
| 5 | 257,429 | **37.37** |
| 8 | 1,542,142 | 26.59 |
| 12 | 2,143,663 | 28.00 |
| 16 | 2,516,197 | **31.97** |
| 18 | **2,903,085** | 28.49 |
| 23 | 1,694,710 | 29.86 |

**Pico 2024:** 6pm (2.90M viajes) - 71% menor que 2015

### Identificación de Patrones Estructurales

#### Horas Pico (consistente todos los años)
1. **6pm-7pm:** Máximo absoluto (fin jornada laboral)
2. **8am-9am:** Pico matutino (inicio jornada)
3. **12pm-2pm:** Pico mediodía (almuerzos, reuniones)

#### Horas Valle
1. **4am:** Mínimo absoluto (Valle nocturno)
2. **3am-5am:** Ventana de menor demanda

#### Paradoja de Tarifas (2020-2025)
- **Madrugada (4-5am):** Tarifas más altas ($32-39)
  - Explicación: Viajes aeropuerto predominan en horas valle
  - Bajo volumen pero alto valor unitario

- **Rush hour (4pm):** Tarifas altas ($32) a pesar de volumen medio
  - Surge pricing por congestión

### Evolución de Tarifas por Hora

**Comparación 2015 vs 2024 (hora 18h - pico):**
- 2015: 10.0M viajes, $15.72 tarifa
- 2024: 2.9M viajes, $28.49 tarifa
- **Cambio:** -71% volumen, +81% tarifa

**Comparación madrugada (hora 4am):**
- 2015: 2.0M viajes, $18.17
- 2024: 236K viajes, $32.41
- **Cambio:** -88% volumen, +78% tarifa

### Hallazgos Estratégicos

1. **Mercado transformado:** Volumen 2024 es 15% del 2015, pero tarifa +85%
2. **Nueva normalidad:** Tarifas premium son estructurales, no temporales
3. **Curva aplanada:** Diferencia pico-valle menor post-COVID
   - 2015: 5.2x (10M vs 2M viajes pico/valle)
   - 2024: 12.3x (2.9M vs 236K)
4. **Oportunidad madrugada:** Alta tarifa ($37), baja competencia
5. **6pm sigue siendo crítico:** A pesar del declive, mantiene ratio 12x vs valle

### Recomendaciones Operacionales

**Asignación de flota 2024:**
- **100% disponibilidad:** 4pm-8pm (horas pico, tarifas altas)
- **60% disponibilidad:** 8am-3pm, 9pm-12am
- **25% disponibilidad:** 1am-5am (tarifa premium compensa bajo volumen)

**Pricing dinámico:**
- Madrugada (4-5am): Mantener tarifas premium ($35-40)
- Rush hours (8am, 5-6pm): Surge pricing justificado
- Horario normal (10am-3pm): Tarifas base

**Incentivos conductores:**
- Bonos para turnos 4-6am (alta tarifa, baja oferta)
- Turnos optimizados: 2pm-10pm captura ambos picos


---

## Conclusiones Generales del Análisis Histórico

### Transformación del Mercado 2015-2025

1. **Declive masivo de volumen:**
   - 2015: 189M viajes (peak histórico)
   - 2020: 17M viajes (COVID)
   - 2024: 29M viajes (nueva normalidad)
   - **Retracción:** -85% desde peak

2. **Aumento estructural de tarifas:**
   - 2015: $15-16 promedio
   - 2024: $27-29 promedio
   - **Incremento:** +81% en 9 años

3. **Cambio en propinas:**
   - 2015: 21.66% (Manhattan)
   - 2019: 26.11% (pre-COVID)
   - 2024: 26.26%
   - **Nuevo estándar:** 26-27% post-digitalización

### Insights Estratégicos Transversales

1. **Concentración geográfica persistente:**
   - Manhattan: 78-82% de ingresos (todos los años)
   - Top 10 zonas: Upper East Side, Midtown, Penn Station (invariantes)

2. **Segmentación clara de mercado:**
   - **Urbano:** Viajes cortos (9-12 min), volumen medio, tarifa estándar
   - **Aeropuertos:** Viajes largos (40-70 min), volumen constante, tarifa premium
   - **Madrugada:** Baja frecuencia, tarifas máximas ($35-40)

3. **Resiliencia de patrones:**
   - Horas pico (6pm, 8am) se mantienen a pesar de COVID
   - Velocidades diurnas vs nocturnas: ratio 2:1 consistente
   - Duración por zona: P50 estable 2015-2025

4. **Calidad sobre cantidad:**
   - Menos viajes pero más rentables
   - Ingreso por viaje aumentó 81%
   - Propinas aumentaron 26%

### Aplicaciones de Negocio por Horizonte

**Inmediato (Q1 2025):**
- Ajustar flota según curva 2024 (no usar referencias 2015)
- Implementar surge pricing en 4-6pm y 4-5am
- ETAs dinámicos por zona usando P50/P90 históricos

**Mediano plazo (2025):**
- Programa de incentivos para conductores madrugada (alta tarifa)
- Alianzas aeropuertos (JFK/LaGuardia = 42M viajes históricos)
- Campañas de propinas en boroughs con gap (Bronx, Queens)

**Estratégico (2025-2027):**
- Pricing basado en valor, no volumen (mercado premium)
- Diversificación geográfica (reducir dependencia Manhattan)
- Predicción ML usando patrones estacionales 2015-2025
- Evaluación de expansión a zonas emergentes vs consolidación en top 10

### Lecciones del Dataset Histórico

1. **COVID marcó antes/después:** 2020 = punto de inflexión irreversible
2. **Competencia ride-sharing:** Declive pre-COVID (2015-2019) es estructural
3. **Pagos digitales transformaron propinas:** Cambio cultural 2019
4. **Zonas premium resisten:** Top 10 mantienen ranking a pesar de declive
5. **Tarifas = nueva ventaja competitiva:** Volumen perdido, margen ganado

### Calidad y Confiabilidad de Datos

- **Completitud:** 826,215,353 registros procesados (2015-2025)
- **Consistencia:** 32/32 tests dbt PASS (100%)
- **Performance:** Clustering mejora 16x en queries analíticas
- **Arquitectura:** Medallion (Bronze/Silver/Gold) garantiza trazabilidad
- **Granularidad:** 1 fila = 1 viaje, permite análisis detallado

---

**Análisis realizado sobre:**  
- Capa GOLD - Star Schema  
- Data Warehouse: Snowflake  
- Transformaciones: dbt 1.8.7  
- Orquestación: Mage AI  

**Periodo:** Enero 2015 - Octubre 2025 (10 años, 10 meses)  
**Total viajes analizados:** 826,215,353  
**Evidencias:** Ver carpeta `evidencias/` para queries y resultados visuales
